# PTSD Mouse Data 

The total data consist of two data sets, measuring responses from 20 different mice at 4 timepoints each: Baseline, Pre-fear exposure, immediately after the Fear exposure, and 9 days later (D9). 10 of these mice are seratonin transporter knockouts (KO) and 10 are wildtype (WT). 

The data are: 
- 79 MRIs. The pre-fear, fear, and D9 images are MN(II) enhanced MRIs (MEMRI). The MEMRI images are used to measure neuronal functioning - (when neurons are active, their uptake of MN(II) is increased?). The baseline images, which are regular fMRI, are 
- 79 measurements of percent time spent in the light by the mice. 

In both data sets, the KO_04_D9 datapoint is missing (hence 79 instead of 80). 

In [95]:
# Import needed packages for analysis
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

import cfl_examples.lesion_mapping.brain_util as BU
import cfl_examples.lesion_mapping.brain_vis as BV

# load response data 
Y = pd.read_pickle('Y.pkl')

# MRI Data 
Some facts about the data: 
- the dimensions of the brain box in each image are (124, 200, 82) (2,033,600 voxels total)
- the images are originally in RPS orientation. We flip them to RAS orientation because then they have the same alignment as some other MRIs we've been looking at 

In [96]:
behav_csv = 'PTSD_Data_Share\Behavior_data\PTSD_PerLight.csv'
mri_dir = 'PTSD_Data_Share\MEMRI_data'

In [97]:
# The images start out in RPS orientation
nib_loaded_img = nib.load(os.path.join(mri_dir, "PTSD_KO_03_BL.nii"))
print(nib.orientations.aff2axcodes(nib_loaded_img.affine))

# load one image to check out its dimensions
img = BU.load_brain(os.path.join(mri_dir, "PTSD_KO_03_BL.nii"))
mri_dims = img.shape
print(mri_dims)

('R', 'P', 'S')
(124, 200, 82)


In [98]:
# load all the images in RAS orientation 
X, Y_unused = BU.load_data(mri_dir, behav_csv, mri_dims, ori='RAS')

In [99]:
#check all images are the same shape 
for fp in os.listdir(mri_dir):
    full_path = os.path.join(mri_dir, fp)
    brain = BU.load_brain(full_path)
    assert brain.shape ==  mri_dims
    #print(brain.shape)

# Load Masks
We were given two masks to fit the the MRIs, a linearly aligned- and non-linearly aligned mask. These masks tell which voxels in the image are part of the brain vs which are empty space. 

The difference between the non-linear and linear mask (from an email from Taylor): "The non-linearly aligned mask is better aligned to the data, however the non-linear "warping" creates artifacts near the surface of the brain that may result in grabbing more undesired non-brain voxels when masking. The linearly aligned mask may not align to the surface of the brain as well and may miss or cut out brain tissue, but may not grab as many non-brain voxels. 

The non-linear mask leaves 531,632 voxels and the linear mask 482,793 voxels unmasked. 

In [100]:
# both masks are binary arrays, containing 1s where there is brain and 0s where there is not 
print(np.unique(nl_mask))
print(np.unique(l_mask))

[0. 1.]
[0. 1.]


In [101]:
# load the non-linear mask template
nl_mask_path = os.path.join('PTSD_Data_Share/templates\MuseTemplate_nonlinear_mask.nii')
nl_mask = BU.load_brain(nl_mask_path)
nolin_mask_vec = BU.flatten(nl_mask)

# load the linear mask template
l_mask_path = os.path.join('PTSD_Data_Share/templates\MuseTemplate_linear_mask.nii')
l_mask = BU.load_brain(l_mask_path)
lin_mask_vec = BU.flatten(l_mask)

In [102]:
print(nl_mask.shape == mri_dims) #non-linear mask has same dims as other MRI images
print(l_mask.shape == mri_dims) #linear mask has same dims as other MRI images

True
True


In [103]:
# how many voxels are unmasked? 
print("Non-linear mask: ", np.sum(nolin_mask_vec==1))
print("Linear mask: ", np.sum(lin_mask_vec==1))

Non-linear mask:  531632
Linear mask:  482793



When we visualize the two masks side-by-side on an MRI below, we indeed see that the non-linear mask has a bumpier edge, with more little islands of disconnected MRI image, while the linear mask has a smoother edge but cuts off some brain tissue. For now, we are using only the non-linearly aligned mask.

In [104]:
# specify labels for plot (note the labels below are specifically for RAS orientation)
dir_labels = { 'saggital' :   ['P', 'A', 'D', 'V'],
               'coronal' :    ['L', 'R', 'D', 'V'],
               'horizontal' : ['L', 'R', 'A', 'P']} 

# create a copy of one image with each mask already applied
nl_masked_brain = X[0].copy()
nl_masked_brain[BU.flatten(nl_mask)==0] = np.nan

l_masked_brain = X[0].copy()
l_masked_brain[BU.flatten(l_mask)==0] = np.nan

# generate interactive plots
BV.plot_interactive_panels(np.vstack((nl_masked_brain, l_masked_brain)), mri_dims, nolin_mask_vec, figsize=(12, 3), std_scale='std', dir_labels=dir_labels, column_titles=["Non-linear Mask", "Linear Mask"])


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

# Click through a bunch of the mice 

The below images compare, side-by-side, MRIs from multiple mice from the same timepoint. Since the first 5 brains in the list were selected, these are all MRIs from KO mice. 

In [105]:
baseline_indices = Y[Y["Timepoint"]=="BL"].index.tolist()
prefear_indices = Y[Y["Timepoint"]=="PreF"].index.tolist()
postfear_indices = Y[Y["Timepoint"]=="Fear"].index.tolist()
d9_indices = Y[Y["Timepoint"]=="D9"].index.tolist() 
timepoints_dir = {'BL' : baseline_indices, 'PreF': prefear_indices, "Fear": postfear_indices, 'D9': d9_indices}


# Show first 5 baseline images 
BV.plot_interactive_panels(X[baseline_indices[:5]], mri_dims, nolin_mask_vec, figsize=(15, 5), colormap="seismic", dir_labels=dir_labels, column_titles=["Mouse 1", "Mouse 2", "Mouse 3", "Mouse 4", "Mouse 5"])

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

Some observations: 
Top (Saggital) Image 
- around slices 32-39, pieces of brain appear and disappear irregularly
- slice 39-40: lot more white in second image than others (and again around 85-89)
- 62 - dot of high activation, high for all except fourth

Middle (Coronal) Image 
- 20-40: noticeable difference in brightness of images (esp bt 1st and 3rd)
- higher brightness in 2 noticeable at 52-54

Botton (Horizontal) Image 
- the lower brightness of 4th brain is again visible (note high activation regions at slices ~16-36) 
- higher brightness of 2nd brain visible around slice ~46 


Do the white dots of high activity (ie those in observations above, also coronal slice 64) have a known meaning? 

In [106]:
# Show first 5 prefear images 
BV.plot_interactive_panels(X[prefear_indices[:5]], mri_dims, nolin_mask_vec, figsize=(15, 5), colormap="seismic", dir_labels=dir_labels, step=1)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

3rd brain brightest, then 4th brain, least bright is 1st

Top (Saggital) Image 
- 39-40: bright spots (same as in baseline images)


Coronal 
- around slice 30-44: the circles in the brain kind of appear differently in different images - the brains may not be perfectly aligned ? 
- 54: bright spot 
- 68: bright spots 
- 80: really bright spots 
- 124: really bright spots


Horizontal
- lots of artifacts visible at beginning
- 17: really bright spots (brainstem connection to brain?)
- some interesting structures toward the end 


- which images are generally brightest/darkest vary between images

In [107]:
# Show first 5 fear images 
BV.plot_interactive_panels(X[postfear_indices[:5]], mri_dims, nolin_mask_vec, figsize=(15, 5), colormap="seismic", dir_labels=dir_labels, step=1)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

Saggital image 
- 39: bright spot
- 54: tiny bright spot at the bottom of the brain 
- 62, 63, 64: abrupt shifts in where the bright/dark patterns are (this is present in the earlier images too but it's clearer in these)

Coronal 
- 30+ : the circles that seemed somewhat misaligned in the pre-fear images are not so in this set 
- 71: really bright spots 
- 77: really bright spots 


Horistonal 
- 16 bright spots 
- 56: on 2nd and 5th brains, the bright teardrop-shaped spots look a little asymetrical (left larger than right)



- All of these brains look generally consistent in brightness


These images generally look brighter, but I think that's just because the brightest brightest activation for this set of images (>25000) is lower than for BL or Pre-F (>30000)

In [172]:
# Show first 5 D9 images 
BV.plot_interactive_panels(X[d9_indices[:5]], mri_dims, nolin_mask_vec, figsize=(15, 5), colormap="seismic", dir_labels=dir_labels, step=1)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

# Heatmap 

These heatmaps show the average value of each voxel for the group 

## Heatmap with each timepoint broken out 

In [173]:
baseline_indices = Y[Y["Timepoint"]=="BL"].index.tolist()
prefear_indices = Y[Y["Timepoint"]=="PreF"].index.tolist()
postfear_indices = Y[Y["Timepoint"]=="Fear"].index.tolist()
d9_indices = Y[Y["Timepoint"]=="D9"].index.tolist() 
timepoints_dir = {'BL' : baseline_indices, 'PreF': prefear_indices, "Fear": postfear_indices, 'D9': d9_indices}

from tqdm import tqdm

# HM_list = [] 
# for timepoint in timepoints_dir: 
#     currentHM = np.zeros(np.prod(mri_dims))
#     indices = timepoints_dir[timepoint]
#     n = len(timepoints_dir[timepoint])
#     for brain in tqdm(X[indices]):
#         currentHM += brain
#     np.divide(currentHM, n)
#     HM_list.append(currentHM)

#     save_name = 'heatmap_' + timepoint + '.npy'
#     np.save(os.path.join(save_name), currentHM) 

100%|██████████| 19/19 [00:00<00:00, 174.18it/s]


In [178]:
# load heatmaps 
heatmap_names = ['heatmap_' + str(timepoint) for timepoint in timepoints_dir.keys()]
HM_list = []
for heatmap in heatmap_names: 
    HM_list.append(np.load(heatmap + '.npy'))

# display heatmaps 
BV.plot_interactive_panels(np.vstack(HM_list), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=list(timepoints_dir))

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

The Baseline images are in general slightly darker than the other images. 

My interpretation of this is that, since the baseline images are not Mn(II) enhanced, the high activity peaks are lower relative to the background level of activity, and since all the images have been normalized along the same scale, the 'low activity' or 'normal' activity levels for the Baseline images have a higher numeric value after normalization compared to  the other images. 

"Overall, the greater in magnitude the voxel intensity value of a contrast enhanced image comparing to each animal's baseline non-contrast enhanced image, corresponds to greater Mn(II) accumulation i.e. neural activity in a particular cluster of voxels."

From this statement, I think they may be saving that we should analyze the intensity in a pre-fear/fear/D9 image minus the intensity in the baseline image, in order to find spikes in activity? 

Since the "artifacts" on the edge of the brain are generally 0 (regardless of whether they're from the baseline or not), it seems like the artifacts end up appearing as "neutral" - the activity is about the same between baseline and fear image 

- is this true or are the artifacts negative? 

- how does this show up with individual images (non heatmaps)?

# Baseline Adjustment

In this section, images with the baseline MRI subtracted off are shown. 

In [113]:
# function to subtract off the baseline 
def adjust_image(X, id, timepoint): 
    '''input: X (array of MRI images), ID of mouse and timepoint to use ("PreF", "Fear", "D9")

    returns baseline-adjusted image
    '''
    # get image for that mouse at that timepoint 
    image_index = Y.loc[(Y.ID==id) & (Y.Timepoint==timepoint)].index[0]
    image = X[image_index]

    # get image for that mouse at baseline 
    baseline_index = Y.loc[(Y.ID==id) & (Y.Timepoint=="BL")].index[0]
    baseline_image = X[baseline_index]

    # subtract baseline from timepoint image 
    adjusted_mri = image - baseline_image

    return adjusted_mrir4

## Baseline-adjusted Images for an individual mouse 



In [129]:
# plot to show the baseline-adjusted images for an individual mouse 
mouse_id = 7

# get image for that mouse at baseline 
baseline_index = Y.loc[(Y.ID==mouse_id) & (Y.Timepoint=="BL")].index[0]
baseline_image = X[baseline_index]

# get other origianl images for mouse
prefear_index = Y.loc[(Y.ID==mouse_id) & (Y.Timepoint=="PreF")].index[0]
postfear_index = Y.loc[(Y.ID==mouse_id) & (Y.Timepoint=="Fear")].index[0]
d9_index = Y.loc[(Y.ID==mouse_id) & (Y.Timepoint=="D9")].index[0]

#examine a single baseline adjusted image (individual images)
all_ims = np.zeros((3, np.prod(mri_dims)))
for i, timepoint in enumerate(["PreF", "Fear", "D9"]): 
    adj_im = adjust_image(X, mouse_id, timepoint)
    adj_im = BU.flatten(adj_im)
    all_ims[i] += adj_im

BV.plot_interactive_panels(np.vstack((baseline_image,  X[prefear_index], all_ims[0],  X[postfear_index], all_ims[1])), mri_dims, nolin_mask_vec, figsize=(17, 5), dir_labels=dir_labels, column_titles=["Baseline", "Pre-fear", "Adj Pre-fear", "Fear", "Adj Fear"])


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [115]:
BV.plot_interactive_panels(np.vstack((baseline_image, X[d9_index], all_ims[2])), mri_dims, nolin_mask_vec, figsize=(12, 5), dir_labels=dir_labels, column_titles=["Baseline", "D9", "Adj D9"])


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

## Heatmap of each timepoint, adjusted by baseline intensity 

Heatmap = mean value for each timepoint across all MRIs 

In [116]:
HM_dir = {'PreF': np.zeros(np.prod(mri_dims)), "Fear": np.zeros(np.prod(mri_dims)), 'D9': np.zeros(np.prod(mri_dims))}
all_mouse_indices = pd.unique(Y["ID"])


for timepoint in HM_dir:
    # for each mouse....
    for mouse_id in all_mouse_indices:
        # get the image associated with that timepoint, subtract off the baseline
        if mouse_id != 1 or timepoint != 'D9': #don't do this for the missing value
            adjusted_mri = adjust_image(X, mouse_id, timepoint)
            # add the result to the correct heatmap entry 
            HM_dir[timepoint] += BU.flatten(adjusted_mri)
    # divide to get the average 
    n = len(timepoints_dir[timepoint])
    np.divide(HM_dir[timepoint], n) 

In [117]:
BV.plot_interactive_panels(np.vstack(list(HM_dir.values())), mri_dims, nolin_mask_vec, figsize=(12, 3), dir_labels=dir_labels, column_titles=list(HM_dir.keys()))


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

# Variance
This plot shows the level of variance between individual voxels in all the MRIs for each timepoint.

In [112]:
var_dir = timepoints_dir.copy()
for timepoint in timepoints_dir: 

    current_var = np.var(X[timepoints_dir[timepoint]], axis=0)
    var_dir[timepoint] = current_var

BV.plot_interactive_panels(np.vstack(var_dir.values()), mri_dims, nolin_mask_vec, figsize=(15, 5), dir_labels=dir_labels, colormap="Reds", step= 1, column_titles=["Baseline Variance", "Pre-fear Variance", "Fear Variance", "D9 Variance"]) 

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

 The variance tends to be highest on the edges of the brain or in small regions inside of the brain. I believe these regions correspond with the regions of high activity seen above in the individual MRIs. The variance seems to be fairly constant across mice. 

# Coarsening

In the following code, we coarsen the mouse MRIs in order to reduce their dimensionality to more manegeable sizes. The coarsening is done by collapsing n x n cubes of voxels into a single voxel, where n is referred to as the 'coarsening factor'. The new single voxel contains the average value of the previous cube of voxels. 

In [135]:
def new_coarsen(x, coarsening_factor, brain_dims):
    ''' coarsens brain by specified amount
    arguments:
        x: 3D array of brain or 1D array of flattened brain
        coarsening_factor: how many original voxels to include in new voxel along a dimension
        brain_dims: array containing [dim0, dim1, dim2] of 3D brain (int array)
    returns:
        new_x: 3D array of coarsened brain
    '''
    # make 3D if flattened
    if x.ndim < 3:
        x = BU.unflatten(x, brain_dims)

    brain_dims = np.array(brain_dims)
    new_dims = np.ceil(brain_dims/coarsening_factor).astype(int)
    new_x = np.zeros(new_dims)
    for i in range(new_dims[0]):
        for j in range(new_dims[1]):
            for k in range(new_dims[2]):
                # select all voxels to contribute to coarsened voxel (i,j,k)
                istart = i*coarsening_factor
                jstart = j*coarsening_factor
                kstart = k*coarsening_factor
                iend = np.min((istart+coarsening_factor, brain_dims[0])) # handle edge case
                jend = np.min((jstart+coarsening_factor, brain_dims[1]))
                kend = np.min((kstart+coarsening_factor, brain_dims[2]))

                sample = x[istart:iend, jstart:jend, kstart:kend]


                # average 
                new_x[i, j, k] = np.average(sample)
    return new_x


## Coarsened Brain of a single mouse 

If we coarsen by a factor of 3, then the resultant brain goes from dimensions (124, 80, 200) and about 2,000,000 voxels in the brain box to: 

In [156]:
coarsening_factor = 3

##Coarsen using new coarsening function 
coarsened_brain = new_coarsen(brain, coarsening_factor, np.array(mri_dims))

#coarsened non linear mask
coarsened_nl_mask = BU.flatten(BU.coarsen(nl_mask, coarsening_factor, np.array(mri_dims)))

print("New dims: ", coarsened_brain.shape)
print("Number of voxels: ", np.prod(coarsened_brain.shape))

New dims:  (42, 67, 28)
Number of voxels:  78792


In [157]:
single_mouse_all_mris = np.zeros((4, np.prod(coarsened_brain.shape)))
for i, brain in enumerate(X[:4]): 
    single_mouse_all_mris[i] = BU.flatten(new_coarsen(brain, coarsening_factor, mri_dims))

BV.plot_interactive_panels(single_mouse_all_mris, coarsened_brain.shape, coarsened_nl_mask, figsize=(17, 5), dir_labels=dir_labels, column_titles=["Coarsened Baseline", "Coarsened Pre-fear", "Coarsened Fear", "Coarsened D9"])


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=41), Output()…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=66), Output()…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=27), Output()…

My thoughts on the coarsened brains: 
- It's certainly easier to click through all the slices in the coarsened version 
- I think that coarsening by a factor of 5 reduces the resolution so much that the images become less useful to look at 
- Coarsening by a factor of 3 reduces the dimensionality of the data (though it is still very high dimensional data), but the images look much better 
- At our current analysis stage, the coarsening process itself is by far the most time consuming process we do, so in that respect the coarsening is not too useful (yet)

 ## Correlation between each voxel of brains and Y (percent time spent in the light)


For each voxel (after coarsening): 
- have 20 pairs of values: MRI (fear - pre-fear)  and Y (fear- pre-fear)
- want to find correlation for ^ for each voxel 
- graph back onto the brain 

In [169]:
n_images = 79

# new array of coarsened brain images 
X_new = np.zeros((n_images, np.prod(coarsened_brain.shape)))
for i in range(n_images): 
    X_new[i] = BU.flatten(new_coarsen(X[i], coarsening_factor, np.array(mri_dims)))

np.save(os.path.join('coarsened3_X.npy'), X_new)

In [170]:
X_new.shape

(79, 78792)

In [187]:
from scipy.stats import pearsonr
X_new = np.load(os.path.join('coarsened3_X.npy'))

Y_diffs = Y.PerLight[timepoints_dir["Fear"]].to_numpy() - Y.PerLight[timepoints_dir["PreF"]].to_numpy()

X_diffs = X_new[timepoints_dir["Fear"]] - X_new[timepoints_dir["PreF"]]

corrs = np.zeros( X_new.shape[1], dtype=np.int16)



for voxel in range(X_new.shape[1]): 
    value = pearsonr(Y_diffs, X_diffs[:, voxel])[0]
    if np.isnan(value): 
        value = 0 
    corrs[voxel] = value

#note: when everything is a 0, pearsonr returns a nan correlation value

In [180]:
np.save(os.path.join("correlations.npy"), corrs)

In [133]:
BV.plot_interactive_panels(corrs, coarsened_brain.shape, coarsened_nl_mask, figsize=(12, 5), dir_labels=dir_labels, std_scale='corr', column_titles=["Correlations for Coarsened Brain"], step=1)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=24), Output()…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=39), Output()…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=16), Output()…

My take on the above graph: 
It's hard to tell if there's anything there. Note that the scale only varies between +- 0.6 (no extremely strong correlations )

In [ ]:
########## Extra stuff 



## Mean and Variance side-by-side

# display heatmaps 
# BV.plot_interactive_panels(np.vstack((HM_list[0], var_dir['BL'], HM_list[1], var_dir['PreF'])), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=["BL Mean", "BL variance", "Pre-fear Mean", "Pre-fear Var"])





## Heatmap of the WT vs KO mice


# WT_indices = Y[Y["Genotype"]=="WT"].index.tolist()
# KO_indices = Y.dropna(how='any', axis=0)[Y["Genotype"]=="KO"].index.tolist() #remove last value bc it doesn't have an associated MRI 

# # create wild type heatmap 
# heatmap_WT = np.zeros(np.prod(mri_dims))
# n = X[WT_indices].shape[0]
# for brain in tqdm(X[WT_indices]):
#     heatmap_WT += brain
# np.divide(heatmap_WT, n)
# np.save(os.path.join('heatmap_WT.npy'), heatmap)


# # create KO heatmap 
# heatmap_KO = np.zeros(np.prod(mri_dims))
# n = X[KO_indices].shape[0]
# for brain in tqdm(X[KO_indices]):
#     heatmap_KO += brain
# np.divide(heatmap_KO, n) 
# np.save(os.path.join('heatmap_KO.npy'), heatmap)


# heatmap_WT = np.load(os.path.join('heatmap_WT.npy'))
# heatmap_KO = np.load(os.path.join('heatmap_KO.npy'))

# BV.plot_interactive_panels(np.vstack((heatmap_WT, heatmap_KO)), mri_dims, nolin_mask_vec, figsize=(10, 5), dir_labels=dir_labels, column_titles=["Wild-type", "Knockout"], step=1)

In [189]:
# save correlations as nii image 
corrs = np.load('correlations.npy')

corrs = BU.unflatten(corrs, (42, 67, 28))
nii_corrs = nib.Nifti1Image(corrs, nib_loaded_img.affine)

nib.save(nii_corrs, os.path.join('PTSDMice_coarsenedCorrelations.nii'))

In [190]:
img = nib.load('PTSDMice_coarsenedCorrelations.nii')
img.aff2ax